# Cable Property Comparison - 6/28/2016
This program reads a set of neuron hoc files from a given directory and spits out a tabular comparison of each neuron's cable properties; these properties include the number of branch points, total cable length, mean path length, and overall tortuosity.

First, here are the required imports:

In [1]:
# Required for system access (utilized below)
import sys
# Required for os access (utilized below)
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),
                             'dependencies'))
# Required to interpret hoc files
from neuron_readExportedGeometry import *
# Required for efficient calculation of mean and standard deviation
import numpy as np
# Required to display and save data in tabular/graphical form
import matplotlib.pyplot as plt

Next, load up a set of neuron hoc files within a directory as a list of geo objects:

In [2]:
# Specify a directory full of hoc files
directory = '/home/cosmo/marderlab/test/'
# Convert the given directory into a list of hoc files
hocs = [(directory + h) for h in os.listdir(directory)]
# Convert the list of hoc files into a list of geo objects
geos = [demoReadsilent(h) for h in hocs]

Number of subgraphs = 3 / size of graphs: [35, 36, 844]
Removed all but largest subgraphs
Soma is filament_999[0].
7 axons are:
['filament_999[431]', 'filament_999[779]', 'filament_999[760]', 'filament_999[680]', 'filament_999[220]', 'filament_999[875]', 'filament_999[20]']
From soma to tips, tortuosity is 1.6 +- 0.9
Number of subgraphs = 1 / size of graphs: [1494]
Soma is filament_999[0].
8 axons are:
['filament_999[1274]', 'filament_999[1244]', 'filament_999[695]', 'filament_999[897]', 'filament_999[207]', 'filament_999[773]', 'filament_999[3]', 'filament_999[353]']
From soma to tips, tortuosity is 1.7 +- 1.0
Number of subgraphs = 1 / size of graphs: [1278]
Soma is filament_999[0].
6 axons are:
['filament_999[1068]', 'filament_999[865]', 'filament_999[28]', 'filament_999[698]', 'filament_999[635]', 'filament_999[590]']
From soma to tips, tortuosity is 1.9 +- 1.6
Number of subgraphs = 2 / size of graphs: [26, 2630]
Removed all but largest subgraphs
Soma is filament_999[0].
7 axons are

Now that we have our list of geo objects ready to go, let's make a parallel list of names:

In [3]:
# Loop through the filenames in directory and split off irrelevant
# information
names = [h.split('_s')[0].split('_f')[0].split('.h')[0].split('_r')[0] for h in os.listdir(directory)]
names

['878_043_GM',
 '878_058_GM',
 '878_045_GM',
 '878_056_GM',
 '878_053_GM',
 '878_041_GM',
 '878_057_GM',
 '878_049_GM',
 '878_061_GM']

Next up, let's make a parallel list containing the number of branch points present in each neuron:

In [4]:
# Count the number of branches, which is equivalent to the number of
# branch points 
branchpoints = [len(g.branches) for g in geos]
branchpoints

[119, 197, 163, 319, 227, 202, 447, 444, 197]

Now, let's make another parallel list containing the total cable length of each neuron:

In [5]:
# More detailed documentation for this code can be found in cable-length-calculator.ipynb
cablelengths = [] # Initialize a list of cablelengths
for geo in geos:
    tips, ends = geo.getTips() # Store all the tip segments in a list, "tips"
                               # Also store the associated ends in "ends"
    find = PathDistanceFinder(geo, geo.soma) # Set up a PDF object for the
                                             # given geo object, anchored at
                                             # the soma
    paths = [find.pathTo(seg) for seg in tips] # List of all paths
    counted = [] # Initialize a list for keeping track of which segments have
                 # already been measured
    cablelength = 0 # Initialize a running total of cable length
    for path in paths: # Sort through each path
        pruned = [seg for seg in path if seg not in counted] # Limit the paths
                                                             # we work with to
                                                             # those which have
                                                             # not already been
                                                             # measured
        forfind = PathDistanceFinder(geo, pruned[0]) # Initialize a PDF
                                                     # anchored at the earliest
                                                     # unmeasured segment
        cablelength += forfind.distanceTo(pruned[-1]) # Add the distance
                                                      # between the anchor and
                                                      # the tip segment to the
                                                      # running total
        for seg in pruned: # Add all of the measured segments to "counted"
            counted.append(seg)
    cablelengths.append(cablelength)
cablelengths

Soma is filament_999[0].
7 axons are:
['filament_999[431]', 'filament_999[779]', 'filament_999[760]', 'filament_999[680]', 'filament_999[220]', 'filament_999[875]', 'filament_999[20]']
Soma is filament_999[0].
8 axons are:
['filament_999[1274]', 'filament_999[1244]', 'filament_999[695]', 'filament_999[897]', 'filament_999[207]', 'filament_999[773]', 'filament_999[3]', 'filament_999[353]']
Soma is filament_999[0].
6 axons are:
['filament_999[1068]', 'filament_999[865]', 'filament_999[28]', 'filament_999[698]', 'filament_999[635]', 'filament_999[590]']
Soma is filament_999[0].
7 axons are:
['filament_999[2510]', 'filament_999[720]', 'filament_999[135]', 'filament_999[327]', 'filament_999[221]', 'filament_999[2508]', 'filament_999[2233]']
Soma is filament_999[0].
7 axons are:
['filament_999[1291]', 'filament_999[485]', 'filament_999[1990]', 'filament_999[18]', 'filament_999[1477]', 'filament_999[1586]', 'filament_999[1540]']
Soma is filament_999[0].
7 axons are:
['filament_999[1429]', 'fi

[3830.5778984167946,
 6971.858765656526,
 5833.550062983639,
 7986.102991086455,
 6555.961803729618,
 6368.205455140606,
 9290.11209111042,
 11708.066754184674,
 5063.735473456966]

Next, let's make yet another parallel list containing the mean path length of each neuron:

In [6]:
# Loops through geos and calculates the mean path length for each neuron
plmeans = [np.mean(g.getProperties()[0]['Path Length']) for g in geos]
plmeans

Soma is filament_999[0].
7 axons are:
['filament_999[431]', 'filament_999[779]', 'filament_999[760]', 'filament_999[680]', 'filament_999[220]', 'filament_999[875]', 'filament_999[20]']
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
Soma is filament_999[0].
8 axons are:
['filament_999[1274]', 'filament_999[1244]', 'filament_999[695]', 'filament_999[897]', 'filament_999[207]', 'filament_999[773]', 'filament_999[3]', 'filament_999[353]']
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
Soma is filament_999[0].
6 axons are:
['filament_999[1068]', 'filame

/home/cosmo/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/cosmo/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[460.42085987999064,
 380.76616879890076,
 574.83028838629627,
 353.81903514032945,
 437.52576565285398,
 429.87766983007708,
 516.13428471447332,
 505.55963215864551,
 321.99165239061773]

Now, let's make a parallel list containing the standard deviation in path length for each neuron:

In [7]:
# Loops through geos and calculates the standard deviation in path
# length for each neuron
plstds = [np.std(g.getProperties()[0]['Path Length']) for g in geos]
plstds

Soma is filament_999[0].
7 axons are:
['filament_999[431]', 'filament_999[779]', 'filament_999[760]', 'filament_999[680]', 'filament_999[220]', 'filament_999[875]', 'filament_999[20]']
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
Soma is filament_999[0].
8 axons are:
['filament_999[1274]', 'filament_999[1244]', 'filament_999[695]', 'filament_999[897]', 'filament_999[207]', 'filament_999[773]', 'filament_999[3]', 'filament_999[353]']
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
Soma is filament_999[0].
6 axons are:
['filament_999[1068]', 'filame

/home/cosmo/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/cosmo/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[127.34838798843164,
 106.06473188988004,
 165.41301251433856,
 133.49617362994826,
 103.64923752990435,
 140.87719825397204,
 154.15688798456196,
 118.48880512197289,
 126.84258289293699]

Next up, let's make another parallel list containing the mean tortuosity of each neuron:

In [8]:
# Loops through geos and calculates the mean tortuosity for each neuron
tortmeans = [np.mean(g.getProperties()[0]['Tortuosity']) for g in geos]
tortmeans

Soma is filament_999[0].
7 axons are:
['filament_999[431]', 'filament_999[779]', 'filament_999[760]', 'filament_999[680]', 'filament_999[220]', 'filament_999[875]', 'filament_999[20]']
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
Soma is filament_999[0].
8 axons are:
['filament_999[1274]', 'filament_999[1244]', 'filament_999[695]', 'filament_999[897]', 'filament_999[207]', 'filament_999[773]', 'filament_999[3]', 'filament_999[353]']
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
Soma is filament_999[0].
6 axons are:
['filament_999[1068]', 'filame

/home/cosmo/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/cosmo/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[1.5564916972920186,
 1.7401888168597806,
 1.8807115845448708,
 2.3045092409787795,
 1.3767677900781505,
 1.849584416451802,
 2.1153574538391546,
 2.4466401412161538,
 2.6102751191805726]

Now, let's make yet another parallel list containing the standard deviation in tortuosity for each neuron:

In [9]:
# Loops through geos and calculates the standard deviation in tortuosity
# for each neuron
tortstds = [np.std(g.getProperties()[0]['Tortuosity']) for g in geos]
tortstds

Soma is filament_999[0].
7 axons are:
['filament_999[431]', 'filament_999[779]', 'filament_999[760]', 'filament_999[680]', 'filament_999[220]', 'filament_999[875]', 'filament_999[20]']
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
Soma is filament_999[0].
8 axons are:
['filament_999[1274]', 'filament_999[1244]', 'filament_999[695]', 'filament_999[897]', 'filament_999[207]', 'filament_999[773]', 'filament_999[3]', 'filament_999[353]']
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
2-node segment
Soma is filament_999[0].
6 axons are:
['filament_999[1068]', 'filame

/home/cosmo/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/cosmo/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[0.89749158629147874,
 0.97395624504218947,
 1.5793366768811326,
 1.9667621973996456,
 0.28716592202598951,
 1.0730427745796853,
 2.231330499621996,
 1.0831052557503111,
 3.165177956875993]

# Making the Table - 6/29/16
Hooray! We now have all of the numbers we set out to calculate! All that's left is to organize them in a table:

In [10]:
filename = 'Test.txt'

f = open(filename, 'w')

props = ['Neuron ID', 'Branches', 'Total Cable Length (um)',
         'Mean Path Length (um)', 'Path Length Standard Deviation',
         'Mean Tortuosity', 'Tortuosity Standard Deviation']

tablebar = \
'+---------------------------------------------------------------------------------+'

f.write('\n' + tablebar + '\n')
f.write('|{0:<26}|{1:<22}|{2:<31}|'.format(props[0], props[1],
                                           props[2]) + '\n')
f.write(tablebar + '\n')
for n in range(len(geos)):
    f.write('|{0:<26}|{1:<22}|{2:<31}|'.format(names[n],branchpoints[n],
                                               cablelengths[n]) + '\n')
    f.write(tablebar + '\n')

f.write('\n' + tablebar + '\n')
f.write('|{0:<26}|{1:<22}|{2:<31}|'.format(props[0], props[3],
                                           props[4]) + '\n')
f.write(tablebar + '\n')
for n in range(len(geos)):
    f.write('|{0:<26}|{1:<22}|{2:<31}|'.format(names[n], plmeans[n],
                                               plstds[n]) + '\n')
    f.write(tablebar + '\n')

f.write('\n' + tablebar + '\n')
f.write('|{0:<26}|{1:<22}|{2:<31}|'.format(props[0], props[5],
                                           props[6]) + '\n')
f.write(tablebar + '\n')
for n in range(len(geos)):
    f.write('|{0:<26}|{1:<22}|{2:<31}|'.format(names[n], tortmeans[n],
                                             tortstds[n]) + '\n')
    f.write(tablebar + '\n')

f.close()

with(open(os.path.join(os.getcwd(), filename))) as fr:
    print(fr.read())


+---------------------------------------------------------------------------------+
|Neuron ID                 |Branches              |Total Cable Length (um)        |
+---------------------------------------------------------------------------------+
|878_043_GM                |119                   |3830.5778984167946             |
+---------------------------------------------------------------------------------+
|878_058_GM                |197                   |6971.858765656526              |
+---------------------------------------------------------------------------------+
|878_045_GM                |163                   |5833.550062983639              |
+---------------------------------------------------------------------------------+
|878_056_GM                |319                   |7986.102991086455              |
+---------------------------------------------------------------------------------+
|878_053_GM                |227                   |6555.961803729618       

Alternatively, we can make the table using matplotlib: